Выбор проекта

Дедлайн - 23:59 8 октября
После дедлайна я закрою таблицу на редактирование

В колонку "Предложение проекта" можете записать свои идеи для проекта. Если вы хотите присоединиться к какой то команде и делать их проект, то можете оставить это поле пустым
Вам важно за неделю разбиться на группы 3-5 человек и вписать в поле "Финальный выбор проекта" название проекта, который вы будете делать и сдавать

Вы должны сделать 2 вещи:
1) К 8 октября у каждого должна быть заполнена графа "Финальный выбор проекта" - это тот проект, который вы будете делать вместе с командой
2) Важно не только определиться с проектом, но и найти людей, с кем вы его будете делать

На проекте могут работать от 3-5 человек

Если вы не выберете проект, то вы не сможете сдать курс, потому что проект составляет 70% оценки за курс

Делать проект в одиночку не рекомендуется

https://docs.google.com/spreadsheets/d/1NVQv2CNGeyoHYbYGBFx0rhv7iDNYAJm8pQBImWeCjwg/edit?usp=sharing

Минимальные требования по реализации проекта

1) У вас должна использоваться LLM/VLM в вашем проекте (можно API, можно локально разворачивать)
2) У вас обязательно в проекте должен быть реализован RAG
3) Ваша задача должна быть решена с высоким качеством. Здесь вы сами решаете как будете замерять качество
4) Презентация проекта и красивое оформление кода по стилю

Описание выше соответствует решению с одним LLM агентом. Но будет приветствоваться, если вы сделаете мультиагентную систему - достаточно будет 2 агентов. Мультиагентность мы будем обсуждать в нашем курсе ближе к ноябрю

От себя добавлю, что если вы хотите разворачивать модель локально, то тут запрета нет. Но учитывайте, что вам нужны мощности для больших моделей, а маленькие модели вряд ли обеспечат вам хорошее качество на выходе

2) Разобраться с API Mistral
3) Разобраться с RAG
4) Придумать метрику качества

In [ ]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
import re
import os
from google.colab import userdata
from mistralai import Mistral

In [ ]:
# Access the secret
api_key = userdata.get('MISTRAL_API_KEY')

if not api_key:
    raise ValueError("❌ MISTRAL_API_KEY not found in Colab secrets!")
else:
    print("✅ API key loaded successfully from Colab secrets!")

# Initialize Mistral client
client_1 = Mistral(api_key=api_key)

# Test connection
def test_connection():
    try:
        models = client.models.list()
        print("✅ Connected successfully!")
        print(f"Available models: {[m.id for m in models.data]}")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        print("💡 If key is not active yet, wait a few minutes and try again")

test_connection()

✅ API key loaded successfully from Colab secrets!
❌ Connection failed: name 'client' is not defined
💡 If key is not active yet, wait a few minutes and try again


<!-- 1) Загрузка pdf-документа в гугл коллаб -->
2) Преобразование pdf-документа (doc, docx) (загрузить в гугл диск) (PYpdf) в текст. $\textbf{Айторе}$


3) препроцессинг

3.1) Реализовать OCR и CLIP. $\textbf{Кирилл}$

3.2) Убирание знаков припинание, рассмотр заглавных букв и прочего для улучшения качества ответов модельки $\textbf{Илья}$

...

4) Векторизация+индексация в RAG (mistral embedding), загрузим в weaviate (что будем хранить?) - (семантику чанка) $\textbf{Алан, Мавджуда}$

 <!-- 6) Получение вопроса от пользователя -->

7) Нахождение релевантных чанков по вектору вопроса (косинусной метрикой) $\textbf{изи}$

 <!-- 8) Использование Mistral ('mistral-large-latest') для ответа (передаем эти чанки как контекст в промпт для Mistral API) -->

 <!-- 9) Вывод ответа -->

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.find('tokenizers/punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


FileSystemPathPointer('/root/nltk_data/tokenizers/punkt_tab')

In [ ]:
!wget https://raw.githubusercontent.com/langchain-ai/langchain/9a277cbe007706b48fa98787ab85a11c59ccba2e/docs/docs/integrations/document_loaders/example_data/layout-parser-paper.pdf

--2025-11-12 09:09:49--  https://raw.githubusercontent.com/langchain-ai/langchain/9a277cbe007706b48fa98787ab85a11c59ccba2e/docs/docs/integrations/document_loaders/example_data/layout-parser-paper.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4686220 (4.5M) [application/octet-stream]
Saving to: ‘layout-parser-paper.pdf’

layout-parser-paper 100%[===================>]   4.47M  18.8MB/s    in 0.2s    

2025-11-12 09:09:50 (18.8 MB/s) - ‘layout-parser-paper.pdf’ saved [4686220/4686220]



In [ ]:
import nltk

from pathlib import Path
from pprint import pprint

from langchain.document_loaders import PyPDFLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive connected.")

Mounted at /content/drive
Google Drive connected.


In [ ]:
PDF_FILE_PATH = "/content/drive/MyDrive/MulRAG.pdf"

In [ ]:
loader = PyPDFLoader(PDF_FILE_PATH, extract_images=True)
pages = loader.load()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pages

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2022-10-21T00:59:50+00:00', 'author': '', 'keywords': '', 'moddate': '2022-10-21T00:59:50+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/drive/MyDrive/MulRAG.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='MuRAG: Multimodal Retrieval-Augmented Generator\nfor Open Question Answering over Images and Text\nWenhu Chen, Hexiang Hu, Xi Chen, Pat Verga, William W. Cohen\nGoogle Research\n{wenhuchen,hexiang,patverga,wcohen}@google.com\nAbstract\nWhile language Models store a massive\namount of world knowledge implicitly in their\nparameters, even very large models often fail\nto encode information about rare entities and\nevents, while incurring huge computational\ncosts. Recently, retrieval-augmented models,\nsuch as REALM, RAG, and RET

In [ ]:
len(pages)

13

In [ ]:
WEAVIATE_API_KEY = "UU02aHp5NFlBMnVHSWRSTV9LakoyNGY1dzJmQzVsaExKYzR5S0VzeVo0TWtxNExyZGZnek5BU0swU3ZZPV92MjAw"
WEAVIATE_CLUSTER = "hhnfg8g8ruijh30yl7jya.c0.europe-west3.gcp.weaviate.cloud"

In [ ]:
import weaviate
from weaviate.auth import AuthApiKey

WEAVIATE_URL = "https://" + WEAVIATE_CLUSTER
auth = AuthApiKey(api_key=WEAVIATE_API_KEY)
client = weaviate.Client(
    url=WEAVIATE_URL,
    auth_client_secret=auth
)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# подготовка к векторизации (будем использовать в Weaviate.from_documents)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

/tmp/ipython-input-2871217763.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Разделение текста на чанки (chunks)
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Создание экземпляра разделителя текста
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Размер каждого чанка в символах
    chunk_overlap=20   # Перекрытие между чанками в символах
)

# Применение разделителя к документам (pages)
docs = text_splitter.split_documents(pages)

In [ ]:
docs

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2022-10-21T00:59:50+00:00', 'author': '', 'keywords': '', 'moddate': '2022-10-21T00:59:50+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/drive/MyDrive/MulRAG.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='MuRAG: Multimodal Retrieval-Augmented Generator\nfor Open Question Answering over Images and Text\nWenhu Chen, Hexiang Hu, Xi Chen, Pat Verga, William W. Cohen\nGoogle Research\n{wenhuchen,hexiang,patverga,wcohen}@google.com\nAbstract\nWhile language Models store a massive\namount of world knowledge implicitly in their\nparameters, even very large models often fail\nto encode information about rare entities and\nevents, while incurring huge computational\ncosts. Recently, retrieval-augmented models,\nsuch as REALM, RAG, and RET

In [ ]:
# ругалось на формат ptex_fullbanner, скорее всего если документов будет много будет плохо(долго) работать
for doc in docs:
    if 'ptex.fullbanner' in doc.metadata:
        doc.metadata['ptex_fullbanner'] = doc.metadata.pop('ptex.fullbanner')


In [ ]:
  # Измерим метрики после сохранения в Weaviate и выберем оптимальные размеры чанков(точность поиска (precision/recall), время запросов, релевантность ответов.)
  #from langchain.text_splitter import RecursiveCharacterTextSplitter

  # Тестируем разные размеры
  #sizes = [200, 500, 1000]
  #for size in sizes:
      #text_splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=int(size*0.1))
      #docs = text_splitter.split_documents(pages)
      # Далее: векторизация и оценка
      #print(f"Размер {size}: {len(docs)} чанков")

In [ ]:
# создание экземпляра векторного хранилища на основе наших документов
from langchain.vectorstores import Weaviate
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

In [ ]:
vector_db.similarity_search("what is rag?", k=3)

[Document(metadata={'author': '', 'creationdate': '2022-10-21T00:59:50Z', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2022-10-21T00:59:50Z', 'page': 9, 'page_label': '10', 'producer': 'pdfTeX-1.40.21', 'ptex_fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'source': '/content/drive/MyDrive/MulRAG.pdf', 'subject': '', 'title': '', 'total_pages': 13, 'trapped': '/False'}, page_content='ral Information Processing Systems, 33:9459–9474.'),
 Document(metadata={'author': '', 'creationdate': '2022-10-21T00:59:50Z', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2022-10-21T00:59:50Z', 'page': 2, 'page_label': '3', 'producer': 'pdfTeX-1.40.21', 'ptex_fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'source': '/content/drive/MyDrive/MulRAG.pdf', 'subject': '', 'title': '', 'total_pages': 13, 'trapped': '/False'}, page_content='fθ(e) and fed to a decoder for t

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
!pip install -qU langchain-mistralai

In [ ]:
from langchain_mistralai import ChatMistralAI

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
model = ChatMistralAI(
    api_key=api_key,
    model="mistral-large-latest",
    temperature=0.7,
    max_tokens=512
)


In [ ]:
output_parser = StrOutputParser() #для чистого вывода ответа
retriever = vector_db.as_retriever() # поисковик на основе наших документов

In [ ]:
rag_chain = (
    {"context": vector_db.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
rag_chain.invoke("what is Multimodal RAG?")

'**Multimodal RAG (MuRAG)** is a retrieval-augmented generation model that extends traditional RAG by incorporating **both visual and textual knowledge** (e.g., images and text) to enhance language generation. Unlike prior text-only RAG methods, MuRAG uses a **multimodal encoder-decoder architecture** (e.g., combining Vision Transformers for images and T5 for text) to retrieve and encode cross-modal information.\n\nIt leverages **pre-trained multimodal transformers** to fuse features from images and text, enabling tasks like multimodal question answering (e.g., VQA) where answers require reasoning over both modalities. The model’s backbone encodes image-text pairs for retrieval and generation, addressing limitations of unimodal systems.'

In [ ]:
rag_chain.invoke("Retell the summary of the article about multimodul RAG")

'The article introduces **MuRAG (Multimodal Retrieval-Augmented Generation)**, the first retrieval-augmented model capable of leveraging **both visual and textual knowledge**, unlike prior text-only models like RAG or FiD.\n\nMuRAG outperforms baselines (e.g., AutoRouting) in multimodal QA tasks, achieving higher **EM (Exact Match) and F1 scores** across text, image, and combined modalities (e.g., 60.8 EM for text vs. 15.4 for question-only baselines). It uses **multimodal transformers** to fuse deep features from vision and language, enabling cross-modal reasoning.\n\nAblation studies show that **pre-training on datasets like LAION** significantly boosts performance, while evaluations highlight strong results in tasks like **VQA (72% accuracy)**, image-text retrieval (85% RECALL@1), and text-based reading comprehension (>55% EM). The model demonstrates **efficiency in multi-task learning** and scalable retrieval from multimodal memory.'

In [ ]:
retriever.invoke("What is Multimodal RAG?")

[Document(metadata={'author': '', 'creationdate': '2022-10-21T00:59:50Z', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2022-10-21T00:59:50Z', 'page': 2, 'page_label': '3', 'producer': 'pdfTeX-1.40.21', 'ptex_fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'source': '/content/drive/MyDrive/MulRAG.pdf', 'subject': '', 'title': '', 'total_pages': 13, 'trapped': '/False'}, page_content='guage modeling, while RAG and FiD adopt an\nencoder-decoder model with a generative language\nmodeling objective. Compared to them, MuRAG\nis the ﬁrst retrieval-augmented model that is ca-\npable of using knowledge presented in multiple\nmodalities (i.e. visual and textual knowledge data),\nwhereas all prior methods are restricted to using\ntext-only knowledge.\nMultimodal Transformers Multimodal trans-\nformers have demonstrated strong performances\nin learning cross-modal representation that are gen-\nerally beneﬁcial on downstream vis

In [ ]:
import re
import cv2
import pytesseract
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

In [ ]:
class Ocr:
    def preprocess_image(self, image_path, scale_percent=200):
        self.scale_percent = scale_percent
        img = cv2.imread(image_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        width = int(gray_img.shape[1] * self.scale_percent / 100)
        height = int(gray_img.shape[0] * self.scale_percent / 100)
        gray_img = cv2.resize(gray_img, (width, height), interpolation=cv2.INTER_CUBIC)
        denoised = cv2.fastNlMeansDenoising(gray_img, h=10)
        return denoised

    def clean_text(self, text):
        cleaned = re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s.,;:!?()%\-—–«»]', '', text)
        return cleaned

    def img2txt(self, img, min_confidence=60):
        self.min_confidence = min_confidence
        processed = self.preprocess_image(img)
        config = '--psm 6 --oem 3'
        data = pytesseract.image_to_data(
            processed,
            lang='rus+eng',
            config=config,
            output_type=pytesseract.Output.DICT
        )
        filtered_text = []
        for i, conf in enumerate(data['conf']):
            if int(conf) > self.min_confidence:
                filtered_text.append(data['text'][i])
        return ' '.join(filtered_text)

    def process(self, image_path):
        text = self.img2txt(image_path)
        return self.clean_text(text)


class ImageCaptioner:
    def __init__(self, model_name="Salesforce/blip-image-captioning-large"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.processor = BlipProcessor.from_pretrained(model_name)
        self.model = BlipForConditionalGeneration.from_pretrained(model_name).to(self.device)

    def describe(self, image_path, max_length=50):
        image = Image.open(image_path)
        inputs = self.processor(images=image, return_tensors="pt").to(self.device)

        output = self.model.generate(**inputs, max_new_tokens=max_length)
        caption = self.processor.decode(output[0], skip_special_tokens=True)

        return caption

    def describe_with_context(self, image_path, question):
        image = Image.open(image_path)
        inputs = self.processor(images=image, text=question, return_tensors="pt").to(self.device)

        output = self.model.generate(**inputs)
        answer = self.processor.decode(output[0], skip_special_tokens=True)

        return answer

In [ ]:
captioner = ImageCaptioner()
description = captioner.describe("The_life_of_Tolstoy.jpg")
print(description)


ocr = Ocr()
text = ocr.process('The_life_of_Tolstoy.jpg')
print(text)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


a close up of a book with a text on it
94 THE LIFE OF TOLSTOY The results he published in his book, A Criticism of Dogmatic Theology. Freeing himself from the creed of the Church, he was inevitably led to examine the teaching of Christianity as contained in the Bible, and conse- quently the Bible itself. He did this in a lengthy work, The Four Gospels Unified and Trans- lated. In this work, step by step, he analysed the text of the Gospels, throwing aside that which was not clear or not directly connected with the main idea of Christianity. The passages clearly express- ing this principal idea he arranged in a connected, easily understood form, and the whole teaching assumed a complete, harmonious, and popular character. Arriving at the very root of Christianity, Tolstoy undertook a new work to explain his con- ception of it: What is My Faith? It may be said that, with this book, the cycle of his religious development was accomplished.
